In [219]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

#import dataset
user_df = pd.read_csv('User_SongFeatures_data.csv')

For our model we will randomly split the dataset into three. There will be two test data sets and one validation dataset. The first data set will be used to perform matrix factorization to extract user and item latent factors. The second dataset will be used to train our classification model. And lastly, our validation set will be used to evaluate our model. 

In [220]:
# split into train, train 2 and validation set
# make sure users who have only listened once is in train set

song_count = user_df.groupby('user_id').count()[['song_id']].reset_index()
one_timers = song_count[song_count['song_id'] == 1]
len(one_timers)

3355

In [221]:
# subset data of users who only listened one time
one_df = user_df[user_df.user_id.isin(one_timers.user_id)]

In [222]:
# subset rest of data
df = user_df[~user_df.user_id.isin(one_timers.user_id)]
len(df)

1139721

In [223]:
X = df.drop('listen_count', axis=1)
y = df.listen_count

In [224]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4, random_state=1)

In [225]:
print(len(X_train))
print(len(X_test))
print(len(X_val))

410299
455889
273533


In [226]:
train1 = X_train.join(y_train).append(one_df, ignore_index=True)
train2 = X_test.join(y_test)
val = X_val.join(y_val)

c:\users\amand\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [227]:
print(len(train1))
print(len(train2))
print(len(val))

413654
455889
273533


In [228]:
# since there are songs with multiple ids with have to group by user id and song and sum the listen counts

train1_df = train1.groupby(['user_id','song'], as_index=False)['listen_count'].sum()
print(train1_df.user_id.nunique())
print(train1_df.song.nunique())

69433
5730


Before we perform non negative matrix factorization on our dataset we must first transform it into a matrix with user_id and song and the corresponding listen count. 

In [229]:
mf_df = train1_df.pivot(index = 'user_id', columns ='song', values = 'listen_count').fillna(0)
mf_df.head()
# https://stackoverflow.com/questions/57370472/recommendation-system-with-matrix-factorization-for-huge-data-gives-memoryerror

song,& Down - Boys Noize,' Cello Song - Nick Drake,'97 Bonnie & Clyde - Eminem,'Round Midnight - Amy Winehouse,'Round Midnight - Miles Davis,(Antichrist Television Blues) - Arcade Fire,(I Just) Died In Your Arms - Cutting Crew,(If You're Wondering If I Want You To) I Want You To - Weezer,(Nice Dream) - Radiohead,(The Symphony Of) Blase' - Anberlin,...,and then patterns - Four Tet,clouding - Four Tet,high fives - Four Tet,in white rooms - Booka Shade,mOBSCENE - Marilyn Manson,paranoid android - Christopher O'Riley,smile around the face - Four Tet,sun drums and soil - Four Tet,the Love Song - K-OS,you were there with me - Four Tet
user_id,,,,,,,,,,,,,,,,,,,,,
00003a4459f33b92906be11abe0e93efc423c0ff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00005c6177188f12fb5e2e82cdbd93e8a3f35e64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00030033e3a2f904a48ec1dd53019c9969b6ef1f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0007235c769e610e3d339a17818a5708e41008d9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000a5c8b4d8b2c98f7a205219181d039edcd4506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [230]:
# turn into matrix

mf_df = mf_df.as_matrix()

c:\users\amand\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


We will now perform non negative matrix factorization to extract latent factors from our matrix.

In [231]:
from sklearn.decomposition import NMF

model = NMF(n_components=7, init='random', random_state=0)
W = model.fit_transform(mf_df)
H = model.components_

# https://stackoverflow.com/questions/57370472/recommendation-system-with-matrix-factorization-for-huge-data-gives-memoryerror

Pick number of components with best score.

In [232]:
W.shape

(69433, 7)

In [233]:
W

array([[2.29228068e-08, 8.67592092e-06, 0.00000000e+00, ...,
        2.89032627e-05, 7.30421687e-07, 7.14826463e-08],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.58918422e-04, 6.17474877e-06, 2.11221576e-07],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.92210821e-01, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.10573248e-06, 1.42774099e-05, 2.80199734e-05, ...,
        6.19839755e-05, 8.47343874e-06, 2.94954799e-06],
       [5.86095724e-06, 1.63583658e-04, 3.62764354e-05, ...,
        4.32615926e-04, 1.60673843e-05, 4.21652039e-06],
       [5.77237798e-07, 3.13436797e-04, 1.55806663e-05, ...,
        8.49770043e-05, 4.69959442e-05, 7.29944211e-06]])

In [234]:
H

array([[3.73600847e-06, 6.79938883e-07, 2.74834595e-07, ...,
        2.56567450e-07, 3.03186699e-07, 7.03629518e-08],
       [6.94515711e-05, 1.55863634e-05, 1.78821950e-05, ...,
        7.50019874e-06, 8.66720493e-04, 2.84011406e-06],
       [0.00000000e+00, 0.00000000e+00, 5.60078809e-07, ...,
        6.65113220e-05, 7.09465288e-06, 1.26295718e-06],
       ...,
       [2.21425535e-03, 3.02677106e-04, 6.05542492e-04, ...,
        2.41360136e-04, 2.54805727e-04, 5.04269483e-05],
       [3.65716851e-02, 7.14558128e-05, 1.18989626e-04, ...,
        3.38875127e-05, 8.05220915e-05, 2.64381162e-05],
       [1.00440182e-04, 3.97826286e-05, 1.15116200e-05, ...,
        1.33479074e-05, 1.32953736e-04, 5.02861851e-06]])

In [235]:
import numpy as np

user = pd.DataFrame(W, columns = ['u1','u2','u3','u4','u5','u6','u7'])

song = pd.DataFrame(H).transpose()


In [236]:
user.head()

,u1,u2,u3,u4,u5,u6,u7
0,2.292281e-08,8.675921e-06,0.000000,0.000055,0.000029,7.304217e-07,7.148265e-08
1,0.000000e+00,0.000000e+00,0.000000,0.000196,0.000159,6.174749e-06,2.112216e-07
2,0.000000e+00,0.000000e+00,0.000000,0.000000,0.492211,0.000000e+00,0.000000e+00
3,4.859820e-08,6.610512e-06,0.000002,0.000011,0.000012,2.942126e-05,1.186700e-05
4,4.055804e-06,9.234415e-07,0.000052,0.000006,0.000018,3.937425e-07,3.131225e-05


In [237]:
train1_df.user_id.unique()

array(['00003a4459f33b92906be11abe0e93efc423c0ff',
       '00005c6177188f12fb5e2e82cdbd93e8a3f35e64',
       '00030033e3a2f904a48ec1dd53019c9969b6ef1f', ...,
       'fffd6a2bdef646ce9898b628d5dd56c43df69a9d',
       'fffd9635b33f412de8ed02e44e6564e3644cf3c6',
       'fffea3d509760c984e7d40789804c0e5e289cc86'], dtype=object)

Now we will append the user and song latent factors found to our original dataset. 

In [238]:
# add latent factors as new columns

user_factors = pd.DataFrame(train1_df.user_id.unique()).join(user)
user_factors.columns = ['user_id','u1','u2','u3','u4','u5','u6','u7']

user_factors.head()


,user_id,u1,u2,u3,u4,u5,u6,u7
0,00003a4459f33b92906be11abe0e93efc423c0ff,2.292281e-08,8.675921e-06,0.000000,0.000055,0.000029,7.304217e-07,7.148265e-08
1,00005c6177188f12fb5e2e82cdbd93e8a3f35e64,0.000000e+00,0.000000e+00,0.000000,0.000196,0.000159,6.174749e-06,2.112216e-07
2,00030033e3a2f904a48ec1dd53019c9969b6ef1f,0.000000e+00,0.000000e+00,0.000000,0.000000,0.492211,0.000000e+00,0.000000e+00
3,0007235c769e610e3d339a17818a5708e41008d9,4.859820e-08,6.610512e-06,0.000002,0.000011,0.000012,2.942126e-05,1.186700e-05
4,000a5c8b4d8b2c98f7a205219181d039edcd4506,4.055804e-06,9.234415e-07,0.000052,0.000006,0.000018,3.937425e-07,3.131225e-05


In [239]:
song_factors = pd.DataFrame(train1_df.song.unique(), columns = ['song']).join(song)
song_factors.columns = ['song','s1','s2','s3','s4','s5','s6','s7']
song_factors.head()


,song,s1,s2,s3,s4,s5,s6,s7
0,Lights Of Ayodhya - Yulara,3.736008e-06,0.000069,0.000000e+00,0.000911,0.002214,0.036572,0.000100
1,Ironmasters - The Men They Couldn't Hang,6.799389e-07,0.000016,0.000000e+00,0.000091,0.000303,0.000071,0.000040
2,Chasing Cars - Snow Patrol,2.748346e-07,0.000018,5.600788e-07,0.000085,0.000606,0.000119,0.000012
3,Secrets - OneRepublic,2.552698e-07,0.000004,1.755655e-04,0.000471,0.001095,0.000043,0.000014
4,You'd Be So Nice To Come Home To - Julie London,9.558672e-07,0.000034,5.485172e-06,0.000166,0.000479,0.000121,0.000252


In [240]:
train2_df = pd.merge(train2, user_factors, on='user_id')

In [241]:
train2_df = pd.merge(train2_df, song_factors, on='song')

In [242]:
len(train2_df)

443454

In [243]:
train2_df.head()

,user_id,song_id,title,release,artist_name,year,song,id,name,uri_x,...,u5,u6,u7,s1,s2,s3,s4,s5,s6,s7
0,f1ccb26d0d49490016747f6592e6f7b1e53a9e54,SODXVXU12AF729E02B,grey room,9,damien rice,2006,Grey Room - Damien Rice,7qSSjZr6Lm9j1sBle4X0b1,grey room,spotify:track:7qSSjZr6Lm9j1sBle4X0b1,...,0.004019,0.000473,0.000329,1.606814e-07,0.000062,0.000029,0.000051,0.001321,0.000018,0.000038
1,e20a32284cd5057d914813261587daef0646902f,SODXVXU12AF729E02B,grey room,9,damien rice,2006,Grey Room - Damien Rice,7qSSjZr6Lm9j1sBle4X0b1,grey room,spotify:track:7qSSjZr6Lm9j1sBle4X0b1,...,0.000102,0.000159,0.000024,1.606814e-07,0.000062,0.000029,0.000051,0.001321,0.000018,0.000038
2,28b191426861422cbfb837d0c0843faf413465cb,SODXVXU12AF729E02B,grey room,9,damien rice,2006,Grey Room - Damien Rice,7qSSjZr6Lm9j1sBle4X0b1,grey room,spotify:track:7qSSjZr6Lm9j1sBle4X0b1,...,0.011262,0.000351,0.000096,1.606814e-07,0.000062,0.000029,0.000051,0.001321,0.000018,0.000038
3,02f988e5c3b8313719d05c2d84d2c6ee70f04212,SODXVXU12AF729E02B,grey room,9,damien rice,2006,Grey Room - Damien Rice,7qSSjZr6Lm9j1sBle4X0b1,grey room,spotify:track:7qSSjZr6Lm9j1sBle4X0b1,...,0.001059,0.000042,0.000051,1.606814e-07,0.000062,0.000029,0.000051,0.001321,0.000018,0.000038
4,bac559ba85b95bad5dfe9bb381a794714aae1ef4,SODXVXU12AF729E02B,grey room,9,damien rice,2006,Grey Room - Damien Rice,7qSSjZr6Lm9j1sBle4X0b1,grey room,spotify:track:7qSSjZr6Lm9j1sBle4X0b1,...,0.000992,0.000043,0.000070,1.606814e-07,0.000062,0.000029,0.000051,0.001321,0.000018,0.000038


In [244]:
train2_df.columns

Index(['user_id', 'song_id', 'title', 'release', 'artist_name', 'year', 'song',
       'id', 'name', 'uri_x', 'acousticness', 'analysis_url', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature',
       'track_href', 'type', 'uri_y', 'valence', 'listen_count', 'u1', 'u2',
       'u3', 'u4', 'u5', 'u6', 'u7', 's1', 's2', 's3', 's4', 's5', 's6', 's7'],
      dtype='object')

In [245]:
df.listen_count.describe()

count    1.139721e+06
mean     2.877347e+00
std      6.173287e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      3.000000e+00
max      2.213000e+03
Name: listen_count, dtype: float64

In [246]:
df.listen_count.quantile(.60)

2.0

Since this is a classification problem, we will transform listen_count into labels of 'one', 'two', 'three', 'four' and 'five_plus'.

In [247]:
def f(row):
    if  row['listen_count'] == 1:
        val = 'one'
    elif row['listen_count'] == 2:
        val = 'two'
    elif row['listen_count'] == 3:
        val = 'three'
    elif row['listen_count'] == 4:
        val = 'four'
    else:
        val ='five_plus'
    return val

In [248]:
train2_df['label'] = train2_df.apply(f, axis=1)

In [249]:
# extract relevant columns for classification

train2_X = train2_df[['acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature','valence','s1', 's2', 's3', 's4', 's5', 's6','s7', 'u1','u2','u3','u4','u5','u6','u7']]

In [ ]:
##SCALE????#
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

In [250]:
len(train2_X)

443454

In [251]:
# extract labels
train2_y = train2_df['label']

In [252]:
# merge user and song latent factors to validation set

val_df = pd.merge(val, user_factors, on='user_id')
val_df = pd.merge(val_df, song_factors, on='song')
val_df['label'] = val_df.apply(f, axis=1)

In [253]:
val_X = val_df[['acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature','valence','s1', 's2', 's3', 's4', 's5', 's6','s7', 'u1','u2','u3','u4','u5','u6','u7']]

In [254]:
val_y = val_df['label']

In [288]:
### VALIDATION TAKING TOO LONG ~400k rows ###
# tune XGBoost for regularization

params = {
        'reg_alpha':[80, 90, 100, 150, 200]}

In [257]:
params2 = {'gamma':[i/10.0 for i in range(0,5)]} ## ignore, accidently deleted random search of this grid

In [289]:
from sklearn.model_selection import RandomizedSearchCV

gs_random = RandomizedSearchCV(XGBClassifier(), params, cv=3 ,n_iter=50)

# subset only 50000 rows for time
gs_random.fit(train2_X[0:50000], train2_y[0:50000])
gs_pred = gs_random.predict(train2_X[0:50000])
print(gs_random.best_params_, gs_random.best_score_)

c:\users\amand\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:271: UserWarning: The total space of parameters 5 is smaller than n_iter=50. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


{'reg_alpha': 200} 0.55032


In [299]:
#XGBOOST TRAINING ACCURACY

from xgboost import XGBClassifier
xgb = XGBClassifier(
 objective= 'multi:softprob', gamma = 0.4, reg_alpha = 500)
xgb.fit(train2_X, train2_y)
xgb_pred = xgb.predict(train2_X)

print('\n Confusion Matrix:\n',confusion_matrix(train2_y,xgb_pred))
print("\n Classification Report: \n", classification_report(train2_y, xgb_pred))
print("Accuracy:", (accuracy_score(train2_y,xgb_pred)))


 Confusion Matrix:
 [[    25      0  64917      0      0]
 [    18      0  18465      0      0]
 [     9      0 257730      0      0]
 [    11      0  31945      0      0]
 [     6      0  70328      0      0]]


c:\users\amand\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\amand\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\amand\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 Classification Report: 
               precision    recall  f1-score   support

   five_plus       0.36      0.00      0.00     64942
        four       0.00      0.00      0.00     18483
         one       0.58      1.00      0.74    257739
       three       0.00      0.00      0.00     31956
         two       0.00      0.00      0.00     70334

   micro avg       0.58      0.58      0.58    443454
   macro avg       0.19      0.20      0.15    443454
weighted avg       0.39      0.58      0.43    443454

Accuracy: 0.5812440523707081


In [300]:
# XGBoost TEST accuracy
from sklearn.metrics import (classification_report,confusion_matrix, accuracy_score, f1_score, roc_auc_score)
xgb_pred2 = xgb.predict(val_X)

print('\n Confusion Matrix:\n',confusion_matrix(val_y,xgb_pred2))
print("\n Classification Report: \n", classification_report(val_y, xgb_pred2))
print("Accuracy:", (accuracy_score(val_y,xgb_pred2)))
print("Accuracy:", (roc_auc_score(val_y,xgb_pred2)))



 Confusion Matrix:
 [[    20      0  39386      0      0]
 [    11      0  11079      0      0]
 [     4      0 154393      0      0]
 [     4      0  19052      0      0]
 [     4      0  42096      0      0]]


c:\users\amand\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\amand\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\amand\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 Classification Report: 
               precision    recall  f1-score   support

   five_plus       0.47      0.00      0.00     39406
        four       0.00      0.00      0.00     11090
         one       0.58      1.00      0.73    154397
       three       0.00      0.00      0.00     19056
         two       0.00      0.00      0.00     42100

   micro avg       0.58      0.58      0.58    266049
   macro avg       0.21      0.20      0.15    266049
weighted avg       0.41      0.58      0.43    266049

Accuracy: 0.580393085484253


ValueError: multiclass format is not supported

Looks like the XGBoost model is overfitting our data. We will tune the hyperparameters (GAMMA, ALPHA, LAMBDA) to avoid this.

In [285]:
from sklearn.ensemble import RandomForestClassifier

rfclf = RandomForestClassifier()
rfclf.fit(train2_X, train2_y)

y_pred = rfclf.predict(val_X)

print('\n Confusion Matrix:\n',confusion_matrix(val_y,y_pred))
print("\n Classification Report: \n", classification_report(val_y, y_pred))
print("Accuracy:", (accuracy_score(val_y,y_pred)))

c:\users\amand\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



 Confusion Matrix:
 [[  7848    386  28487    531   2154]
 [  1554    134   8517    162    723]
 [ 11513    933 133770   1483   6698]
 [  2273    193  15144    310   1136]
 [  4173    322  34748    542   2315]]

 Classification Report: 
               precision    recall  f1-score   support

   five_plus       0.29      0.20      0.24     39406
        four       0.07      0.01      0.02     11090
         one       0.61      0.87      0.71    154397
       three       0.10      0.02      0.03     19056
         two       0.18      0.05      0.08     42100

   micro avg       0.54      0.54      0.54    266049
   macro avg       0.25      0.23      0.22    266049
weighted avg       0.43      0.54      0.46    266049

Accuracy: 0.5426707110344335


In [202]:
from sklearn.neighbors import KNeighborsClassifier

Knnclf = KNeighborsClassifier(n_neighbors=3)
Knnclf.fit(train2_X, train2_y)

knn_predictions = Knnclf.predict(val_X)

print('\n Confusion Matrix:\n',confusion_matrix(val_y,knn_predictions))
print("\n Classification Report: \n", classification_report(val_y, knn_predictions))
print("Accuracy:", (accuracy_score(val_y, knn_predictions)))


 Confusion Matrix:
 [[ 12147   1740  21989    722   2808]
 [  2805    611   6600    233    841]
 [ 28283   6007 107652   2260  10195]
 [  4413    979  11835    389   1440]
 [  9125   1906  27206    739   3124]]

 Classification Report: 
               precision    recall  f1-score   support

   five_plus       0.21      0.31      0.25     39406
        four       0.05      0.06      0.05     11090
         one       0.61      0.70      0.65    154397
       three       0.09      0.02      0.03     19056
         two       0.17      0.07      0.10     42100

   micro avg       0.47      0.47      0.47    266049
   macro avg       0.23      0.23      0.22    266049
weighted avg       0.42      0.47      0.44    266049

Accuracy: 0.46579013640344447


In [177]:
print("Random Forest:", "%.2f%%" %(accuracy_score(val_y,y_pred)*100))
print("KNN:", "%.2f%%" %(accuracy_score(val_y,knn_predictions)*100))
print("XGBoost:", "%.2f%%" %(accuracy_score(val_y,xgb_pred)*100))

Random Forest: 70.48%
KNN: 68.40%
XGBoost: 73.91%


In [178]:
print("Random Forest:", (f1_score(val_y,y_pred, average='weighted')))
print("KNN:", (f1_score(val_y,knn_predictions, average='weighted')))
print("XGBoost:", (f1_score(val_y,xgb_pred, average='weighted')))

Random Forest: 0.6818852759683732
KNN: 0.6628205516707145
XGBoost: 0.6317749552271772


Tune random forest

In [296]:
param_grid_new = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [297]:
from sklearn.model_selection import RandomizedSearchCV

cv_new = RandomizedSearchCV(rfclf, param_grid_new, n_iter = 100, cv = 3)
cv_new.fit(train2_X[0:1000], train2_y[0:1000])
print(cv_new.best_params_, cv_new.best_score_)

{'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 70, 'bootstrap': True} 0.481


In [298]:
from sklearn.ensemble import RandomForestClassifier

rfclf = RandomForestClassifier(n_estimators = 200, min_samples_split = 10, min_samples_leaf = 4, max_features = 'auto', max_depth = 70, bootstrap= True)
rfclf.fit(train2_X, train2_y)

y_pred = rfclf.predict(val_X)

print('\n Confusion Matrix:\n',confusion_matrix(val_y,y_pred))
print("\n Classification Report: \n", classification_report(val_y, y_pred))
print("Accuracy:", (accuracy_score(val_y,y_pred)))


 Confusion Matrix:
 [[  2914      1  36409     12     70]
 [   404      0  10653      4     29]
 [  1206      0 153025     10    156]
 [   474      0  18514     19     49]
 [   617      0  41383      3     97]]

 Classification Report: 
               precision    recall  f1-score   support

   five_plus       0.52      0.07      0.13     39406
        four       0.00      0.00      0.00     11090
         one       0.59      0.99      0.74    154397
       three       0.40      0.00      0.00     19056
         two       0.24      0.00      0.00     42100

   micro avg       0.59      0.59      0.59    266049
   macro avg       0.35      0.21      0.17    266049
weighted avg       0.49      0.59      0.45    266049

Accuracy: 0.5865648809053972


In [ ]:
## now retune estimators, min split and min leaf
# look into changing number of classes and number of latent factors

Now we will write a function that will print out top 10 songs for a user

In [182]:
def get_top_songs(user_id):
    
    # get songs user has listened to 
    listened_songs = train2_df[train2_df.user_id == user_id].song.unique()
    
    # get songs user has not listened to 
    songs = train2_df.drop(columns=['user_id','listen_count']).drop_duplicates('song')
    not_listened = songs[~songs.song.isin(listened_songs)].drop(columns=['u1', 'u2', 'u3', 'u4', 'u5', 'u6', 'u7','label'])
    not_listened['user_id'] = user_id
    
    # join user features and song features on songs not listened to
    not_listened_df = not_listened.merge(user_factors, on = 'user_id')
    
    # run classifier on songs not listened to
    pred = xgb.predict_proba(not_listened_df[['acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature','valence','s1', 's2', 's3', 's4', 's5','s6','s7', 'u1','u2','u3','u4','u5','u6','u7']])
    
    # add probabilities of belonging to class 'high_count'
    not_listened_df['pred']= pred[:,0]
    
    # get top 10 predictions
    top_pred = not_listened_df.sort_values(by ='pred', ascending=False).head(10)
    return top_pred[['song_id', 'song', 'pred']]

In [ ]:
## maybe change to top 5 songs????

In [183]:
get_top_songs('f1ccb26d0d49490016747f6592e6f7b1e53a9e54')

,song_id,song,pred
4974,SOMMIXC12A6D4F9EF0,Collecting Skylines - Swearing At Motorists,0.545943
4341,SOMBGAL12AB0181F7F,(iii) - The Gerbils,0.526405
4889,SOIYGXO12A679D7E64,Taja's Lude (Interlude) - Blackstreet,0.449230
55,SOSXLTC12AF72A7F54,Revelry - Kings Of Leon,0.448057
2,SOAWMKQ12A8C13C340,Make Gay Love Not War - Need New Body,0.410568
1012,SOLMEOG12A8C142046,Naked - Marques Houston,0.405738
5676,SOZQKZS12A8C13B190,A Primera Vista - Pedro Aznar,0.390117
574,SOVHKJL12AB017E2B2,Superballs - Insane Clown Posse,0.387892
4702,SOJLYEB12A6D4F9750,Angel - Matt Nathanson,0.387383
201,SOUFTBI12AB0183F65,Invalid - Tub Ring,0.380381


In [176]:
train2_df[train2_df.user_id == 'f1ccb26d0d49490016747f6592e6f7b1e53a9e54'].sort_values(by='listen_count', ascending=False)[['song','listen_count','label']]

,song,listen_count,label
1323,Sweet home Alabama - Lynyrd Skynyrd,11,high_count
989,Woods - Bon Iver,6,high_count
1841,Who Can Compare - Foolish Things,6,high_count
1196,Lonelily - Damien Rice,4,high_count
2459,Over You - Roxy Music,4,high_count
862,Cannonball - Damien Rice,4,high_count
812,Cheers Darlin' - Damien Rice,4,high_count
1027,Amie - Damien Rice,3,high_count
1288,The Lighthouse's Tale - Nickel Creek,3,high_count
1706,Older Chests - Damien Rice,3,high_count
